In [1]:
interp.repositories() ++= Seq(coursierapi.MavenRepository.of(
"https://jitpack.io"
))

In [2]:
import $ivy. `org.carbonateresearch::picta:0.1`

import $ivy.$                                  

In [3]:
import picta.Html._ // required to initialize jupyter notebook mode
init_notebook_mode() // stops ugly output

import picta.Html._ // required to initialize jupyter notebook mode


# 1. Basics

The aim of the Picta library is to be a highly configurable and composable general purpose charting library. The following examples are aimed at highlighting the charting grammar for the library, which should make constructing charts easy and intuitive.

#### Colors

Colors in the library are represented as one of two types:

1. `List[String]` => this may either be a color word, such as "red", or RGB value, such as "rgb(255, 255, 255, 1)".
2. `List[Double]`

#### Lines

Lines are perhaps the simplest chart component. The constructor takes in the following arguments:

1. `width: Double`
2. `color: List[Double or String]`




They may be composed as follows:

In [4]:
import picta.options.Line

val line = Line() + List("rgb(255, 255, 255, 1)") // the default constructor for Line has a width of 0.5

println(line)

Line(0.5,Some(List(rgb(255, 255, 255, 1))))


#### Markers

Markers take in the following optional parameters:

1. `symbol: String`
2. `color: List[String or Double]`
3. `line: Line`

Again, composition of a line follows a similar pattern. Here we create a `circle` marker, with the color `red` and a `line` of width 0.5:

In [5]:
import picta.options.Marker

val marker = Marker() + "circle" + List("red") + Line()

println(marker)

Marker(None,Some(List(red)),Some(Line(0.5,Some(List()))))


### [TO BE CONTINUED.....]

## Chart compositions

Charts consist of 3 components:

1. `data`
2. `layout`
3. `config`

Generally, most of the compositions make use of `data` and `layout` components.

#### Scatter Charts and Line Charts

In [6]:
// some dummy data for x and y variables
val x = List.range(1, 50)
val y = x.map(x => x + scala.util.Random.nextDouble()*100)

In [6]:
import picta.series.XY

// the two data series are then wrapped inside 
val data = XY(x, y, trace_name="data0", trace_type=SCATTER, trace_mode=Some("markers+lines"))

cmd6.sc:156: object XY is not a member of package picta.series
possible cause: maybe a semicolon is missing before `object XY`?
import picta.series.XY
       ^cmd6.sc:4: not found: value XY
val data = XY(x, y, trace_name="data0", trace_type=SCATTER, trace_mode=Some("markers+lines"))
           ^cmd6.sc:4: not found: value trace_name
val data = XY(x, y, trace_name="data0", trace_type=SCATTER, trace_mode=Some("markers+lines"))
                    ^cmd6.sc:4: not found: value trace_type
val data = XY(x, y, trace_name="data0", trace_type=SCATTER, trace_mode=Some("markers+lines"))
                                        ^cmd6.sc:4: not found: value trace_mode
val data = XY(x, y, trace_name="data0", trace_type=SCATTER, trace_mode=Some("markers+lines"))
                                                            ^Compilation Failed

: 

In [ ]:
// The layout is a composable object which determines how the chart is displayed
val layout = Layout(title=Some("Chart"))
// the configuration option allows us to set whether the chart is responsible and zoomable using scroll
val config = Config(responsive=false, scrollZoom=true)

In [ ]:
// we can compose a chart using the data, layout and config elements above. This grammar is the same for all charts
val chart = Chart() + data + layout + config
chart.plot_inline()

In [ ]:
// additional traces can simply be composed with an existing chart and added on
val data1 = XY(x, y.map(x => x + scala.util.Random.nextDouble()*100), trace_name="data1", trace_type=BAR)

val chart1 = chart + data1
chart1.plot_inline()

## 1.1 Pie Chart

In [ ]:
// pie charts can be composed in a similar way:
val data = XY(List(19, 26, 55), List("Residential", "Non-Residential", "Utility"), trace_name="test", trace_type=PIE)
val layout = Layout(Some("Pie Chart"))
val chart = Chart() + data +  layout + config
chart.plot_inline()

## 3. Histogram + Axes

In [ ]:
val x = List.range(1, 1000).map(_+scala.util.Random.nextDouble()*1000)

// for histogram charts, the xkey must be specified. It essentially sets the orientation (x direction or y direction)
val data = XY(x=x, xkey="x", trace_name="trace4", trace_type=HISTOGRAM)

In [ ]:
// axis can be set just like any other object
val xaxis = Axis(key="xaxis", title = "my x data")
val yaxis = Axis(key="yaxis", title = "my y data")

// these are added to the layout object
val layout = Layout(Some("Histogram with axes")) + xaxis + yaxis

val chart = Chart() + data + layout  + config
chart.plot_inline()

In [ ]:
// we can also compose customizations in much the same way:
val marker = Marker(color=Some(ColorString("rgba(255, 100, 102, 0.4)")), line=Some(Line()))

// change xkey to y to get a horizontal histogram
val data = XY(x, xkey="y", trace_name="test", trace_type=HISTOGRAM, marker=marker)

val layout = Layout(Some("XY.Histogram.Color"))
val chart = Chart() + data + layout + config
chart.plot_inline()

## 4. 2D Histogram Contour

In [ ]:
val x = List.range(1, 50)
val y = x.map(x => x + scala.util.Random.nextDouble()*100)

val data = XY(x, y, trace_name="trace3", trace_type=HISTOGRAM2DCONTOUR, trace_mode=Some("marker"))
val layout = Layout(title=Some("2D Histogram Contour"))

val chart = Chart() + data + layout + config
chart.plot_inline()

## 5. Contour

In [ ]:
val x = List.range(1, 100)
val y = List.range(1, 100)
val z = List.range(1, 100).map(x => x + scala.util.Random.nextDouble()*100)

val data = XYZ(x, y, z, trace_name="trace4", trace_type=CONTOUR, trace_mode=Some("marker"))

In [ ]:
val layout = Layout(title=Some("Contour"))

val chart = Chart() + data + layout + config
chart.plot_inline()

## 6. Heatmap

In [ ]:
val z = List.range(1, 101).map(e => e + scala.util.Random.nextDouble()*100).grouped(10).toList

In [ ]:
// add lines in between the grid items
val data = XYZ(z, trace_name="trace", trace_type=HEATMAP)
val layout = Layout(title=Some("Contour"), height = 500, width = 500)

val chart = Chart() + data + layout + config
chart.plot_inline()

## 7. Scatter3D

In [ ]:
val x = List.range(1, 100)
val y = List.range(1, 100)
val z = List.range(1, 100).map(e => e + scala.util.Random.nextDouble()*100)

val data = XYZ(x, y, z, trace_name="trace", trace_type = SCATTER3D)

val chart = Chart() + data + layout + config
chart.plot_inline()

## 8. Surface Plot

In [ ]:
// 3d surface plot
val k = List(
    List(8.83,8.89,8.81,8.87,8.9,8.87),
    List(8.89,8.94,8.85,8.94,8.96,8.92),
    List(8.84,8.9,8.82,8.92,8.93,8.91),
    List(8.79,8.85,8.79,8.9,8.94,8.92),
    List(8.79,8.88,8.81,8.9,8.95,8.92),
    List(8.8,8.82,8.78,8.91,8.94,8.92),
    List(8.75,8.78,8.77,8.91,8.95,8.92),
    List(8.8,8.8,8.77,8.91,8.95,8.94),
    List(8.74,8.81,8.76,8.93,8.98,8.99),
    List(8.89,8.99,8.92,9.1,9.13,9.11),
    List(8.97,8.97,8.91,9.09,9.11,9.11),
    List(9.04,9.08,9.05,9.25,9.28,9.27),
    List(9,9.01,9,9.2,9.23,9.2),
    List(8.99,8.99,8.98,9.18,9.2,9.19),
    List(8.93,8.97,8.97,9.18,9.2,9.18)
  )

val layout = Layout(Some("Surface Chart"))
val data = XYZ(k, trace_name="trace", trace_type=SURFACE)

val chart = Chart() + data + layout + config
chart.plot_inline()

## 9. Third Dimension as Color

In [ ]:
// call this series name, plot type, change mode 
// create an object - trace mode as string in overloaded constructor

val marker = Marker() + ColorList(z)
val data = XY(x, y, trace_name = "my line", trace_type = SCATTER, trace_mode = Some("lines")) + marker
val chart = Chart() + Layout(Some("Color2D.Basic")) + config + data
chart.plot_inline()

## 10. Grid

In [ ]:
// 1. first we define the grid layout - 1 row, 2 columns
val grid = Grid(1, 2)

// 2. Now define the axes we want to place on the grid
val ax1 = Axis("xaxis", title = "x axis 1")
val ax2 = Axis("xaxis2", title = "x axis 2")

// 3. define the traces
val trace1 = XY(x, y, trace_name="trace1", trace_type=SCATTER, trace_mode=Some("markers"))
val trace2 = XY(x, z, trace_name="trace2", trace_type=SCATTER, trace_mode=Some("markers"), xaxis="x2", yaxis="y2")

// 4. combine into a layout
val layout = Layout(title=Some("Axis Composition")) + List(ax1, ax2) + grid

// 5. construct into a chart
val chart = Chart() + List(trace1, trace2) + layout + config

chart.plot_inline()

## 11. Map

In [ ]:
val line = Line(width = 2, color = Some("red"))
val trace = Map(List(40.7127, 51.5072), List(-74.0059, 0.1275), trace_mode = Some("lines")) + line

val geo = Geo(landcolor = Some("rgb(204, 204, 204)"), lakecolor=Some("rgb(255, 255, 255)")) + 
LatAxis(List(20, 60)) + LongAxis(List(-100, 20))

val layout = Layout() + geo
val chart = Chart() + trace + layout + config
chart.plot_inline()